In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("hf-data") \
    .master("spark://spark-master:7077") \
    .config("spark.eventLog.enabled", "true") \
    .config("spark.eventLog.dir", "file:///opt/spark/spark-events/") \
    .config("spark.executor.memory", "4g") \
    .config("spark.executor.cores", 1) \
    .config("spark.dynamicAllocation.enabled", "true") \
    .config("spark.dynamicAllocation.shuffleTracking.enabled", "true") \
    .config("spark.dynamicAllocation.initialExecutors", 0) \
    .config("spark.dynamicAllocation.minExecutors", 1) \
    .config("spark.dynamicAllocation.maxExecutors", 15) \
    .config("spark.sql.execution.arrow.pyspark.enabled", "true") \
    .config("spark.scheduler.mode", "FAIR") \
    .getOrCreate()

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
24/09/28 17:58:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
24/09/28 17:58:11 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.
24/09/28 17:58:12 WARN Utils: spark.dynamicAllocation.initialExecutors less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update your configs.
24/09/28 17:58:12 WARN Utils: spark.executor.instances less than spark.dynamicAllocation.minExecutors is invalid, ignoring its setting, please update your configs.


In [2]:
df = spark.read.csv("hfdata/ipmaxmind_out_2023_old/", sep='\t', header=True)

In [3]:
df.show()

+---------------+--------------------+--------------------+-----------+--------+---------+---------------+--------------+--------------+----------------+---------------+--------------------+----------------+----------------+----------+----------------+----+
|             ip|                host|              server|postal_code|latitude|longitude|accuracy_radius|continent_code|continent_name|country_iso_code|   country_name|subdivision_iso_code|subdivision_name|       city_name|metro_code|       time_zone|year|
+---------------+--------------------+--------------------+-----------+--------+---------+---------------+--------------+--------------+----------------+---------------+--------------------+----------------+----------------+----------+----------------+----+
| 43.198.191.223|https://www.zysj....|               nginx|       NULL| 22.2842| 114.1759|            100|            AS|          Asia|              HK|      Hong Kong|                NULL|            NULL|       Hong Kong|  

In [5]:
df.repartition(10).write.mode("append").parquet("hfdata/ipmaxmind_out_2023")

In [6]:
import os
from huggingface_hub import HfApi
from dotenv import load_dotenv

/usr/local/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
load_dotenv()

api = HfApi(token=os.environ['HF_API_TOKEN'])

In [8]:
api.upload_folder(
    folder_path="./hfdata/ipmaxmind_out_2023/",
    repo_id="big-banyan-tree/BBT_CommonCrawl_2023",
    repo_type="dataset",
    path_in_repo="ipmaxmind_out",
    ignore_patterns=".*",
);